In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft
import numpy as np

In [ ]:
point_x_dimensions = (
    np.asarray(((0,), (1,), (2,))),
    np.asarray(((0, -1), (1, 0), (2, 4))),
    np.asarray(((0, -1, 8), (1, 0, 16), (2, 4, 24))),
)

## get_probability

In [ ]:
for point_x_dimension in point_x_dimensions:

    print("=" * 80)

    print(point_x_dimension)

    grid_nd, grid_nd_probabilities = kraft.probability.get_probability(
        point_x_dimension
    )

    print(grid_nd.shape)

    print(grid_nd_probabilities.shape)

## get_posterior_probability

In [ ]:
for point_x_dimension in point_x_dimensions:

    print("=" * 80)

    print(point_x_dimension)

    (
        grid_nd,
        grid_nd_posterior_probabilities,
    ) = kraft.probability.get_posterior_probability(point_x_dimension)

    print(grid_nd.shape)

    print(grid_nd_posterior_probabilities.shape)

In [ ]:
for point_x_dimension in point_x_dimensions[1:]:

    print("=" * 80)

    print(point_x_dimension)

    (
        grid_nd,
        grid_nd_posterior_probabilities,
    ) = kraft.probability.get_posterior_probability(
        point_x_dimension, target_dimension_number=2.02
    )

    print(grid_nd.shape)

    print(grid_nd_posterior_probabilities.shape)

## plot_nomogram